# SQL Murder Mystery

### THE PLOT :
There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a ​murder​ that occurred sometime on ​Jan.15, 2018​ and that it took place in ​SQL City​. Start by retrieving the corresponding crime scene report from the police department’s database.

![alt text](schema.png)

In [1]:
# import what we need 
import pandas as pd
import sqlite3


In [2]:
# making a sqllite database with the db file for project

conn = sqlite3.connect('/Users/glenn/Documents/Visual code workspace/Data Science/Projects/SQL_murder/sql-murder-mystery.db')

c = conn.cursor()

In [3]:
# We want to make a querry out of the info we got in plot text 

querry1 = """
select * 
from crime_scene_report
WHERE type = "murder"
AND CITY = "SQL City"
AND date = 20180115

"""

In [4]:
# Set the maximum number of characters displayed so we can see the whole description
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
murders_in_sqlcity = pd.read_sql_query(querry1, conn)
murders_in_sqlcity


,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


OK, we now have more information, we need to make a querry to find the witnesses of the murder. i consentrate on the witness with a known first name

In [5]:
querry2 = """
select * 
from person
WHERE address_street_name = "Franklin Ave"
AND name LIKE "%Annabel%"

"""

In [6]:
# Set the maximum number of characters displayed so we can see the whole description
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
witness1 = pd.read_sql_query(querry2, conn)
witness1


,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


We have now found our first witness Annabel Miller, now we need to find the second witness

In [7]:
# We know his adress_street_name is Northwestern Dr, we need to find the max address_number
querry3 = """
SELECT * 
FROM person
WHERE address_street_name = "Northwestern Dr"
AND address_number = (SELECT MAX(address_number) FROM person WHERE address_street_name = "Northwestern Dr")
"""

In [8]:
# Dataframe of witnes2 
# Set the maximum number of characters displayed so we can see the whole description
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
witness2 = pd.read_sql_query(querry3, conn)
witness2

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


OK, we have found our second witness, now we need to combine the two witnesses to one dataframe


In [9]:
witnesses = pd.concat([witness1, witness2])
witnesses

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


To find the murder we need to look in to the interview to see if the witness said anything about the murder. We then make a new Querry to see what our witness said about the murder

In [10]:
querry4 = """
SELECT 
    interview.transcript,
    person.name,
    person.id
FROM interview
JOIN person
ON interview.person_id=person.id
WHERE person.name= "Annabel Miller"
OR person.name = "Morty Schapiro"
"""


In [11]:
# Make a dataframe out of what our witness said about the murder in the interview
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
interview = pd.read_sql_query(querry4, conn)
interview

,transcript,name,id
0,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W"".",Morty Schapiro,14887
1,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.",Annabel Miller,16371


### Clues so far 
We now know that the murder took place on Jan.15, 2018 and that it took place in SQL City. We also know that the Suspect was a man, He is most likley a Gold member at the gym because he has a bag that just gold members And he have a car with licence include H42W. And the suspect has also been at the gym at january the 9th.
### Clues to follow : 
* find goldmembers at the gym and sort on members with 48Z in it
* Find License plates that have H42W in it
* Cheek out if some of the suspect has been at the gym also at january the 9th

In [12]:
# We do a new querry to find the gold members at the gym that have 48Z in it
querry5 = """
SELECT *
FROM get_fit_now_member
WHERE ID LIKE "%48Z%"
AND membership_status = "gold"
"""

In [13]:
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
suspects = pd.read_sql_query(querry5, conn)
suspects

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


Nice, we now have narrowed our suspects down to 2, now we need to find if the license plates have H42W in it for some of the suspects and also if the suspects have cheeked in at the gym at the date of the murder and also if the suspect has been there at the gym at january the 9th

In [14]:
querry6 = """
SELECT *
FROM drivers_license
where plate_number LIKE "%H42W%"
AND gender = "male"
"""

In [15]:
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
suspects_licence_plate = pd.read_sql_query(querry6, conn)
suspects_licence_plate

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


We got some more information about the suspects, now we need to find if the suspects have been at the gym

In [16]:
# We do a new querry to find out if the suspects have been at the gym
querry7 = """
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180115
AND membership_id IN ("48Z7A", "48Z55")
"""
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
suspects_been_at_gym = pd.read_sql_query(querry7, conn)
suspects_been_at_gym


,membership_id,check_in_date,check_in_time,check_out_time


Hmm , no one of the suspects have cheeked in ordinary at the gym , we update the querry and see if one of them have been there at the gym at january the 9th

In [17]:
querry8 = """
SELECT *
FROM get_fit_now_check_in
WHERE check_in_date = 20180109
AND membership_id IN ("48Z7A", "48Z55")
"""
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
suspects_been_at_gym = pd.read_sql_query(querry8, conn)
suspects_been_at_gym

,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


Not so much help , bouth the suspects have been at the gym at january the 9th, maybe if we can be any smarter if we see when the witnes was at the gym at january the 9th

In [18]:
# We see when the witnes was at the gym at january the 9th, but need to find her membership id first
querry9= """
SELECT *
FROM get_fit_now_member
JOIN get_fit_now_check_in
ON get_fit_now_member.id = get_fit_now_check_in.membership_id
WHERE name = "Annabel Miller"
AND check_in_date = 20180109
"""
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
witnes_at_gym = pd.read_sql_query(querry9, conn)
witnes_at_gym

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,90081,16371,Annabel Miller,20160208,gold,90081,20180109,1600,1700


OK, then we see our witness has been at the gym at january the 9th at the same time as both of the suspects, so we need to find out if just one of them has been there at the day of murder, ur hope then is in the facebook event.

In [19]:
querry10 = """
SELECT *
FROM facebook_event_checkin
JOIN get_fit_now_member ON facebook_event_checkin.person_id = get_fit_now_member.person_id
WHERE date LIKE '20180115'
AND get_fit_now_member.id IN('48Z55', '48Z7A')
"""
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
suspect_at_gym_murder_date = pd.read_sql_query(querry10, conn)
suspect_at_gym_murder_date

,person_id,event_id,event_name,date,id,person_id,name,membership_start_date,membership_status
0,67318,4719,The Funky Grooves Tour,20180115,48Z55,67318,Jeremy Bowers,20160101,gold


### Suspect is placed at the murder site
We can now see that the suspect is placed at the murder site, from the two suspects, the suspect named Jeremy Bowers has been at the gym at the same time as the murder, he has a car registered with the letters H42W in it, and he has been at the gym at january the 9th whwere also the witness saw him.
The other suspect has not been at the gym at the same time as the murder take place.

# Jeremy Bowers is the  Suspected murder.

We need to arrest Jeremy Bowers, so we need to find his adress and all other personal information

In [20]:
# We do a new querry to get all information posible on Jeremy 
querry11 = """
SELECT * 
FROM person
JOIN drivers_license
ON person.license_id = drivers_license.id
JOIN income
ON person.ssn = income.ssn
WHERE name = "Jeremy Bowers"
"""
The_murder = pd.read_sql_query(querry11, conn)
The_murder

,id,name,license_id,address_number,address_street_name,ssn,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model,ssn,annual_income
0,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS,871539279,10500


In [21]:
querry12 = """
SELECT value FROM solution;
"""
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
solution = pd.read_sql_query(querry12, conn)
solution



,value
0,"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."


OK, so it is more to our plot than what we thoutgt, we need to read the Transcript from the murder. 

In [22]:
querry13 = """
SELECT * FROM interview
WHERE person_id = 67318
"""
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
transcript_murder = pd.read_sql_query(querry13, conn)
transcript_murder

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


Then we have more information about the suspect that order the murder, we can make a new querry and use the information from the murder to find the right person in one more querry


In [24]:
querry14 = """ 
SELECT event_name, date, person_id, name, height, annual_income, car_make, person_id, hair_color
FROM facebook_event_checkin
JOIN person
ON facebook_event_checkin.person_id = person.id
JOIN drivers_license
ON person.license_id = drivers_license.id
JOIN income
ON person.ssn = income.ssn
WHERE event_name LIKE "Sql%"
AND date LIKE "%201712__"
AND drivers_license.hair_color = "red"
AND drivers_license.gender = "female"
AND drivers_license.car_make = "Tesla"
"""
pd.set_option('display.max_colwidth', None)
# Make a dataframe out of the querry
murder_hire = pd.read_sql_query(querry14, conn)
murder_hire

,event_name,date,person_id,name,height,annual_income,car_make,person_id,hair_color
0,SQL Symphony Concert,20171206,99716,Miranda Priestly,66,310000,Tesla,99716,red
1,SQL Symphony Concert,20171212,99716,Miranda Priestly,66,310000,Tesla,99716,red
2,SQL Symphony Concert,20171229,99716,Miranda Priestly,66,310000,Tesla,99716,red


# Miranda Priestly is the one how has hired the murderer
We have found out that she attended the SQL Symhony consert 3 times in desember 2017, she has a red hair and a Tesla car, she has high annual income and she is a female, and she match the height